<p align="center">
  <a>
    <img src="./figures/logo-hi-paris-retina.png" alt="Logo" width="280" height="180">
  </a>

  <h3 align="center">Data Science Bootcamp</h3>
</p>

Authors : Yann Berthelot, Florian Bettini, Laure-Amélie Colin

### Resources

For more details about Data Cleaning, please refer to the `1_fires_preparation.ipynb` notebook

Objective of this lab
======

Merge external data (temperature, rainfall, demographics) into a single table. Clean the data to obtain a final dataset, without errors, duplicates, irrelevant values... ready to be analyzed


Objectives of this Notebook
======

##### **Objectives:**
- Read multiple external datasets
    - temperatures and precipitations from the `./data/1_raw/cities/` folder
    - demographics from the file `./data/1_raw/cities/us-cities-demographics.csv`
- Merge them into a unique table
- Save the cleaned DataFrame in `./data/2_clean/external_data.csv`


##### **Data structure**:
- Temperature and precipitation
    - The `./data/1_raw/cities/` folder contains temperature and precipitation values for 210 US cities.
    - it contains a file `./data/1_raw/cities/city_infos.csv` that maps the city name with a code (example: "USW00094728" for "New York")
    - a README file `./data/1_raw/cities/README.txt` can give you additional information on these files.
    - all other files are named with a city code (example: `./data/1_raw/cities/USW00094728.csv` for "New York") and contains historical temperature and precipitations between **1894** and **2021**, if available. They contains the following columns:
        * `Date`: day (format YYYY-mm-dd)
        * `tmax`: maximum temperature observed during the day (in fahrenheit)
        * `tmin`: minimum temperature observed during the day (in fahrenheit)
        * `prcp`: daily precipitation amount (inch per square meter, in/m²)
- Demographics
    - The `./data/1_raw/demographics/us-cities-demographics.csv` file contains demographic data (age, total population, etc.) for US cities. Data are taken in 2015. Columns are:
        * `City`: city name
        * `State`: state name (full name)
        * `Median Age`: median age of the population
        * `Male Population`: male population (integer)
        * `Female Population`: female population (integer)
        * `Total Population`: total population (integer)
        * `Number of Veterans`: number of veterans (integer)
        * `Foreign-born`: number of foreign-born (integer)
        * `Average Household Size`: average size of an household
        * `State Code`: state code (abbreviation)
        * `Race`: most represented race
        * `Count`: unknown column, should not be used

##### One can find bellow some guidelines for this process:
- For cities information (temperature, precipitation, latitude, longitude and city name)
    - read a unique file with temperatures and precipitations, and filter the dates between 2011 and 2015 (included)
    - when reading this unique file, add a column with the city attributes (latitude, longitude and city name)
    - apply this process to read all files, and concatenate all DataFrames into a single one
- For demographics:
    - read the input file and keep one record per city
    - map states' abbreviations from their fullname (with `STATE2ABBREV`)
- For external data:
    - merge the 2 previously created DataFrame from cities and demographics, to get a unique output DataFrame
    - For each dataset, compute a new column with the city name, transformed without spaces and with lowercase. You can use the following function when computing this column: `"".join(city_name.lower().split())`
    - Use this column when joining the 2 DataFrames

<p align="center">
  <a>
    <img src="./figures/UpToYou.png" alt="Logo" width="200" height="280">
  </a>
</p>

#### Libraries

In [1]:
import pandas as pd
from utils import check_duplicates

#### Input variables

In [2]:
checks = {True:"OK", False: "NOK"}

# Dict to be used to plot on states on map, it only recognizes 2 letter codes.
STATE2ABBREV = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

### Step 1. Merge all csv files into a single table

In [4]:
def clean_raw_weather(city_name:str, info_cities:pd.DataFrame, date_min:str, date_max:str)-> pd.DataFrame:
    '''
    Creates a dataset of the daily max temperature, min temperature, precipitation
    with added info of city lat, lon and name
    
    Input:
    city_name (str) : Name of the city to process (case insensitive)
    info_cities (pd.DataFrame) : DataFrame with the city information
    date_min (str) : minimum date for filtering
    date_max (str) : maximum date for filtering
    
    Output:
    (pd.DataFrame) : Processed dataframe
    
    '''
    file_name = info_cities[info_cities["Name"].apply(lambda x : x.lower())==city_name.lower()].ID.iloc[0]
    df = pd.read_csv(f"./data/1_raw/cities/{file_name}.csv", index_col=0, parse_dates=["Date"])
    # Add attributes
    df["Lat"] = info_cities[info_cities["ID"]==file_name].iloc[0]["Lat"]
    df["Lon"] = info_cities[info_cities["ID"]==file_name].iloc[0]["Lon"]
    df["City_name"] = info_cities[info_cities["ID"]==file_name].iloc[0]["Name"]

    # filter on max and min dates
    df = df.loc[(df["Date"] <= date_max)&(df["Date"] >= date_min)]
    
    return df

def prepare_weather(info_cities:pd.DataFrame, date_min:str, date_max:str)->pd.DataFrame:
    '''
    Concatenate the different cities dataframe into a single dataframe sorted by date and city name.
    Filter between min and max dates.
    
    Input:
    info_cities (pd.DataFrame) : DataFrame with the city information
    date_min (str) : minimum date for filtering
    date_max (str) : maximum date for filtering
    
    Output:
    (pd.DataFrame) : Processed dataframe
    
    '''
    city_names = info_cities["Name"].unique()
    weather_timeseries = [clean_raw_weather(city_name, info_cities, date_min, date_max) for city_name in city_names]
    all_cities = pd.concat(weather_timeseries, axis=0).sort_values(by=["Date","City_name"])
    return all_cities

def prepare_demography(df:pd.DataFrame)->pd.DataFrame:
    '''
    Clean the demography dataset
    
    Input:
    df (pd.DataFrame) : Demographics dataset
    
    Output:
    (pd.DataFrame) : Processed dataframe
    
    '''
    # Drop useless demographic data
    df = df[["City", "State", "Median Age", "Total Population", "Average Household Size"]]
    # Since we do not care about racial demographics, we can drop the multiline
    df = df.groupby('City').first()
    # compute state Abbreviation
    df["STATE_CODE"] = df["State"].map(STATE2ABBREV)
    return df

def get_merge_name(names:pd.Series)->pd.Series:
    '''
    Uniformize city names for simpler merging by lowering and removing all whitespaces.
    
    Input:
    df (pd.Series) : Raw city names
    
    Output:
    (pd.Series) : "Clean" city names
    
    '''
    return names.apply(lambda x : "".join(x.lower().split())).values

In [5]:
# inputs
date_min = "2011-01-01"
date_max = "2015-12-31"

# read files
info_cities = pd.read_csv("./data/1_raw/cities/city_info.csv", index_col=0)
demography_raw = pd.read_csv("./data/1_raw/demographics/us-cities-demographics.csv", delimiter=";")

# cleaning temperature and precipitations
all_cities = prepare_weather(info_cities, date_min, date_max)

# cleaning demographics
demography = prepare_demography(demography_raw)

# merge external data into a single dataframe
all_cities["MergeName"] = get_merge_name(all_cities.City_name)
demography["MergeName"] = get_merge_name(pd.Series(demography.index))
external_data = pd.merge(left=all_cities, right=demography, how="left", on=["MergeName"]).drop(columns=["MergeName"])

# check duplicates
c = checks.get(check_duplicates(external_data, ["City_name","Date"]), False)
print(f"Check duplicates: {c}")

# save to csv format
external_data.to_csv("./data/2_clean/external_data.csv", index=False)

Check duplicates: OK


# Take Away
- Finding external data in a AI project can improve the scoring of the model
- It is often necessary to transform this external data before being able to merge it

### Pitfalls to avoid
- not checking for any duplicate values after a join/merge operation